## Introduction

Access to clean and functional water sources is a fundamental necessity, yet many regions in Tanzania struggle to provide reliable water points for their population of over 67 million. Despite numerous water points being established across the country, a significant number are either non-functional or in need of repair. This situation hampers access to clean water, leading to health challenges, time lost in fetching water, and increased burdens on communities, particularly women and children.

This project seeks to address this issue by building a machine learning model to classify the operational status of water points. By analyzing data related to water point features such as pump type, installation details, and geographic characteristics, the project aims to predict whether a water point is functional, functional but requiring repair, or non-functional. Such insights can empower stakeholders to take targeted and proactive actions to improve water access.

## Problem Statement
In Tanzania, many water points fail to provide reliable access to clean water, impacting millions of people and burdening rural communities. As a data scientist, my goal is to develop a machine learning model to predict the operational status of water points—functional, in need of repair, or non-functional. This will enable stakeholders to prioritize maintenance, allocate resources effectively, and address systemic issues, ensuring sustainable access to clean water for the population.

## Objectives
### General Objectives
To create a predictive model that identifies the operational status of water points in Tanzania to enhance water access and support efficient resource allocation.

### Specific Objectives
> Predict Water Point Functionality: Develop a machine learning models to classify water points as functional, functional but in need of repair, or non-functional.

>Evaluate Model Performance: Assess the accuracy, precision, recall, and overall effectiveness of the models to ensure reliable predictions.

> Identify Failure Patterns: Analyze the data to uncover factors and trends contributing to water point failures.

> Optimize Resource Allocation: Provide actionable insights for stakeholders to prioritize maintenance and repair efforts effectively.

## Data Understanding and Loading
The dataset originates from Taarifa and the Tanzanian Ministry of Water and was downloaded from DrivenData https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/.  The following are set of information about the waterpoints:

amount_tsh - Total static head (amount water available to waterpoint)

date_recorded - The date the row was entered

funder - Who funded the well

gps_height - Altitude of the well

installer - Organization that installed the well

longitude - GPS coordinate

latitude - GPS coordinate

wpt_name - Name of the waterpoint if there is one

basin - Geographic water basin

subvillage - Geographic location

region - Geographic location

region_code - Geographic location (coded)

district_code - Geographic location (coded)

lga - Geographic location

ward - Geographic location

population - Population around the well

public_meeting - True/False

recorded_by - Group entering this row of data

scheme_management - Who operates the waterpoint

scheme_name - Who operates the waterpoint

permit - If the waterpoint is permitted

construction_year - Year the waterpoint was constructed

extraction_type - The kind of extraction the waterpoint uses

extraction_type_group - The kind of extraction the waterpoint uses

extraction_type_class - The kind of extraction the waterpoint uses

management - How the waterpoint is managed

management_group - How the waterpoint is managed

payment - What the water costs

payment_type - What the water costs

water_quality - The quality of the water

quality_group - The quality of the water

quantity - The quantity of water

quantity_group - The quantity of water

source - The source of the water

source_type - The source of the water

source_class - The source of the water

waterpoint_type - The kind of waterpoint

waterpoint_type_group - The kind of waterpoint

In [58]:
#Important Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression

In [59]:
train_values = pd.read_csv("Train_values.csv")
train_values.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,3/14/2011,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,3/6/2013,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2/25/2013,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,1/28/2013,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,7/13/2011,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [60]:
train_labels = pd.read_csv("Train_labels.csv")
train_labels.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [61]:
test_values = pd.read_csv("Test_values.csv")
test_values.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [62]:
train_df = train_values.merge(train_labels, on= 'id')
train_df.head(10)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,3/14/2011,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,3/6/2013,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2/25/2013,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,1/28/2013,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,7/13/2011,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
5,9944,20.0,3/13/2011,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,0,...,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,functional
6,19816,0.0,10/1/2012,Dwsp,0,DWSP,33.362410,-3.766365,Kwa Ngomho,0,...,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,non functional
7,54551,0.0,10/9/2012,Rwssp,0,DWE,32.620617,-4.226198,Tushirikiane,0,...,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,non functional
8,53934,0.0,11/3/2012,Wateraid,0,Water Aid,32.711100,-5.146712,Kwa Ramadhan Musa,0,...,salty,salty,seasonal,seasonal,machine dbh,borehole,groundwater,hand pump,hand pump,non functional
9,46144,0.0,8/3/2011,Isingiro Ho,0,Artisan,30.626991,-1.257051,Kwapeto,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional


id, wpt_name, recorded_by and num_private

In [63]:
train_df.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'status_group'],
      dtype='object')

In [64]:
train_df.shape

(59400, 41)

In [65]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [66]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,59400.0,37115.131768,21453.128371,0.00000,18519.750000,37061.500000,55656.500000,7.424700e+04
amount_tsh,59400.0,317.650385,2997.574558,0.00000,0.000000,0.000000,20.000000,3.500000e+05
gps_height,59400.0,668.297239,693.116350,-90.00000,0.000000,369.000000,1319.250000,2.770000e+03
longitude,59400.0,34.077427,6.567432,0.00000,33.090347,34.908743,37.178387,4.034519e+01
latitude,59400.0,-5.706033,2.946019,-11.64944,-8.540621,-5.021597,-3.326156,-2.000000e-08
num_private,59400.0,0.474141,12.236230,0.00000,0.000000,0.000000,0.000000,1.776000e+03
region_code,59400.0,15.297003,17.587406,1.00000,5.000000,12.000000,17.000000,9.900000e+01
district_code,59400.0,5.629747,9.633649,0.00000,2.000000,3.000000,5.000000,8.000000e+01
population,59400.0,179.909983,471.482176,0.00000,0.000000,25.000000,215.000000,3.050000e+04
construction_year,59400.0,1300.652475,951.620547,0.00000,0.000000,1986.000000,2004.000000,2.013000e+03


# Preprocessing


#### Data cleaning

In [67]:
train_df_copied = train_df.copy()

In [68]:
train_df.duplicated().sum()

0

In [69]:
# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(train_df['status_group'])


In [70]:
# Define Features (X) and Target (y)
X = train_df.drop(columns=['status_group', 'id'])  # Drop target and ID columns
y = y_encoded  # Target variable
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## X_Train Data

In [71]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47520 entries, 3607 to 56422
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             47520 non-null  float64
 1   date_recorded          47520 non-null  object 
 2   funder                 44644 non-null  object 
 3   gps_height             47520 non-null  int64  
 4   installer              44631 non-null  object 
 5   longitude              47520 non-null  float64
 6   latitude               47520 non-null  float64
 7   wpt_name               47520 non-null  object 
 8   num_private            47520 non-null  int64  
 9   basin                  47520 non-null  object 
 10  subvillage             47224 non-null  object 
 11  region                 47520 non-null  object 
 12  region_code            47520 non-null  int64  
 13  district_code          47520 non-null  int64  
 14  lga                    47520 non-null  object 
 15 

In [72]:
X_train.isna().sum()

amount_tsh                   0
date_recorded                0
funder                    2876
gps_height                   0
installer                 2889
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 296
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            2689
recorded_by                  0
scheme_management         3102
scheme_name              22523
permit                    2439
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_group                0
quantity

In [73]:
#Checking the percentage of missing values in each column
missing_percentage = X_train.isnull().mean() * 100
missing_percentage[missing_percentage > 0]

funder                6.052189
installer             6.079545
subvillage            0.622896
public_meeting        5.658670
scheme_management     6.527778
scheme_name          47.396886
permit                5.132576
dtype: float64

In [74]:
# dropping column Scheme_name , missing data is almost 50% 
X_train = X_train.drop(columns= ['scheme_name'])

In [75]:
#filling all non-numerical columns values having null values with mode
X_train[X_train.select_dtypes(include=[object]).columns] = X_train.select_dtypes(include=[object]).fillna(X_train.mode().iloc[0])

In [76]:
X_train.isna().sum().sum()

0

In [77]:
X_train.shape

(47520, 38)

In [78]:
X_train.columns

Index(['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin',
       'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward',
       'population', 'public_meeting', 'recorded_by', 'scheme_management',
       'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [79]:
# dropping columns which are not necessary in our model prediction
X_train = X_train.drop(columns= ['wpt_name', 'num_private','recorded_by', 'date_recorded', 'subvillage', 'lga' ,'ward'])

In [80]:
X_train.shape

(47520, 31)

In [81]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47520 entries, 3607 to 56422
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             47520 non-null  float64
 1   funder                 47520 non-null  object 
 2   gps_height             47520 non-null  int64  
 3   installer              47520 non-null  object 
 4   longitude              47520 non-null  float64
 5   latitude               47520 non-null  float64
 6   basin                  47520 non-null  object 
 7   region                 47520 non-null  object 
 8   region_code            47520 non-null  int64  
 9   district_code          47520 non-null  int64  
 10  population             47520 non-null  int64  
 11  public_meeting         47520 non-null  bool   
 12  scheme_management      47520 non-null  object 
 13  permit                 47520 non-null  bool   
 14  construction_year      47520 non-null  int64  
 15 

In [82]:
X_train.select_dtypes(exclude=["int64", "float64"]).columns

Index(['funder', 'installer', 'basin', 'region', 'public_meeting',
       'scheme_management', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

#### Onehotencoding

In [83]:
X_train_categorical = X_train.select_dtypes(exclude=["int64", "float64"])
ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

X_train_ohe = pd.DataFrame(
    ohe.fit_transform(X_train_categorical),
    #index is important to ensure we can concatenate with other columns
    index=X_train_categorical.index,
    # we are dummying multiple columns at once, so stack the names
    columns=np.hstack(ohe.categories_)
)
X_train_ohe

c:\Users\Hp\anaconda3\envs\learn-env\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,A/co Germany,Aar,Abasia,Abc-ihushi Development Cent,Abd,Abdul,Abood,Abs,Aco/germany,...,dam,hand pump,improved spring,other,cattle trough,communal standpipe,dam,hand pump,improved spring,other
3607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
50870,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
20413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
52806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
50091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
38158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
15795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Normalization

In [84]:
X_train_numerical = X_train.select_dtypes(include=["int64", "float64"])
scaler = MinMaxScaler()

X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_numerical),
    # index is important to ensure we can concatenate with other columns
    index=X_train_numerical.index,
    columns=X_train_numerical.columns
)
X_train_scaled

,amount_tsh,gps_height,longitude,latitude,region_code,district_code,population,construction_year
3607,0.000143,0.760678,0.878073,0.637112,0.204082,0.0125,0.005246,0.992548
50870,0.000000,0.022238,0.880156,0.508598,0.000000,0.0750,0.000000,0.000000
20413,0.000000,0.022238,0.805545,0.220458,0.112245,0.0750,0.000000,0.000000
52806,0.000000,0.022238,0.844227,0.242006,0.112245,0.0875,0.000000,0.000000
50091,0.000857,0.383339,0.917896,0.481453,0.040816,0.0125,0.003934,0.992052
...,...,...,...,...,...,...,...,...
54343,0.002857,0.137663,0.901399,0.246765,0.040816,0.0500,0.008361,0.991058
38158,0.002857,0.637487,0.855902,0.161367,0.102041,0.0625,0.001148,0.987581
860,0.000000,0.017649,0.966024,0.534671,0.030612,0.0625,0.032787,0.996026
15795,0.000000,0.022238,0.850574,0.733278,0.163265,0.0750,0.000000,0.000000


### Concatenation

In [85]:
X_train_df = pd.concat([X_train_scaled, X_train_ohe], axis=1)
X_train_df

,amount_tsh,gps_height,longitude,latitude,region_code,district_code,population,construction_year,0,A/co Germany,...,dam,hand pump,improved spring,other,cattle trough,communal standpipe,dam,hand pump,improved spring,other
3607,0.000143,0.760678,0.878073,0.637112,0.204082,0.0125,0.005246,0.992548,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
50870,0.000000,0.022238,0.880156,0.508598,0.000000,0.0750,0.000000,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
20413,0.000000,0.022238,0.805545,0.220458,0.112245,0.0750,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
52806,0.000000,0.022238,0.844227,0.242006,0.112245,0.0875,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
50091,0.000857,0.383339,0.917896,0.481453,0.040816,0.0125,0.003934,0.992052,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54343,0.002857,0.137663,0.901399,0.246765,0.040816,0.0500,0.008361,0.991058,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
38158,0.002857,0.637487,0.855902,0.161367,0.102041,0.0625,0.001148,0.987581,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
860,0.000000,0.017649,0.966024,0.534671,0.030612,0.0625,0.032787,0.996026,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
15795,0.000000,0.022238,0.850574,0.733278,0.163265,0.0750,0.000000,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### LOGISTIC REGRESSION MODEL

In [86]:
print("X_train_scaled column types:")
print(X_train_scaled.columns.map(type))
print("X_train_ohe column types:")
print(X_train_ohe.columns.map(type))


X_train_scaled column types:
Index([<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>],
      dtype='object')
X_train_ohe column types:
Index([<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>,
       ...
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>],
      dtype='object', length=3793)


In [87]:
if X_train_df.columns.duplicated().any():
    print("Duplicate column names found!")
    print(X_train_df.columns[X_train_df.columns.duplicated()])

Duplicate column names found!
Index([                    '0',            'Action Aid',
       'Action Contre La Faim',                  'Adra',
                          'Af',                'Africa',
               'Africaone Ltd',              'Africare',
                'Afroz Ismail',                 'Amref',
       ...
                      'spring',               'unknown',
                         'dam',                 'other',
               'cattle trough',    'communal standpipe',
                         'dam',             'hand pump',
             'improved spring',                 'other'],
      dtype='object', length=498)


In [88]:
# Rename duplicate columns
X_train_df.columns = [
    f"{col}_{i}" if X_train_df.columns.duplicated()[i] else col
    for i, col in enumerate(X_train_df.columns)
]


In [89]:
if X_train_df.columns.duplicated().any():
    print("Duplicate column names found!")
    print(X_train_df.columns[X_train_df.columns.duplicated()])

In [90]:
type(X_train_df)

pandas.core.frame.DataFrame

In [91]:
type(y_train)

numpy.ndarray

In [92]:
y_train.shape

(47520,)

In [93]:
print(y_train.dtype)  # Should be numeric or categorical (e.g., int, float)
print(pd.Series(y_train).isnull().sum())  # Check for null values


int32
0


In [94]:
print(pd.Series(y_train).unique())


[0 2 1]


In [95]:
print(X_train_df.shape)  # Should be (47520, n_features)
print(y_train.shape)     # Should be (47520,)

(47520, 3801)
(47520,)


In [96]:
X_train_df.columns = X_train_df.columns.astype(str)

In [97]:
print(X_train_df.dtypes.value_counts())

float64    3801
dtype: int64


In [98]:
logreg = LogisticRegression(fit_intercept=False, C=1e12, solver='liblinear')
model_log = logreg.fit(X_train_df, y_train)
model_log

LogisticRegression(C=1000000000000.0, fit_intercept=False, solver='liblinear')

## Model Evaluation

In [99]:
y_hat_train = logreg.predict(X_train_df)

train_residuals = np.abs(y_train - y_hat_train)
print(pd.Series(train_residuals, name="Residuals (counts)").value_counts())
print()
print(pd.Series(train_residuals, name="Residuals (proportions)").value_counts(normalize=True))

0    37147
2     7212
1     3161
Name: Residuals (counts), dtype: int64

0    0.781713
2    0.151768
1    0.066519
Name: Residuals (proportions), dtype: float64


X_Test Data

In [100]:
X_train.columns

Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'basin', 'region', 'region_code', 'district_code',
       'population', 'public_meeting', 'scheme_management', 'permit',
       'construction_year', 'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group', 'payment',
       'payment_type', 'water_quality', 'quality_group', 'quantity',
       'quantity_group', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [101]:
X_test.columns

Index(['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin',
       'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward',
       'population', 'public_meeting', 'recorded_by', 'scheme_management',
       'scheme_name', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [102]:
X_train_ohe.shape

(47520, 3793)

In [103]:
X_test_ohe.shape

(11880, 2013)

In [104]:
X_train_scaled.shape

(47520, 8)

In [105]:
# dropping columns which are not necessary in our model prediction or has missing values above 50%
X_test = X_test.drop(columns= ['wpt_name', 'num_private','recorded_by', 'date_recorded', 'subvillage', 'lga' ,'ward', 'scheme_name'])
#filling all non-numerical columns values having null values with mode
X_test[X_test.select_dtypes(include=[object]).columns] = X_test.select_dtypes(include=[object]).fillna(X_test.mode().iloc[0])
#onehotcoding for categorical columns
X_test_categorical = X_test.select_dtypes(exclude=["int64", "float64"])
ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

X_test_ohe = pd.DataFrame(
    ohe.fit_transform(X_test_categorical),
    #index is important to ensure we can concatenate with other columns
    index=X_test_categorical.index,
    # we are dummying multiple columns at once, so stack the names
    columns=np.hstack(ohe.categories_)
)
#Normalization
X_test_numerical = X_test.select_dtypes(include=["int64", "float64"])
scaler = MinMaxScaler()

X_test_scaled = pd.DataFrame(
    scaler.fit_transform(X_test_numerical),
    # index is important to ensure we can concatenate with other columns
    index=X_test_numerical.index,
    columns=X_test_numerical.columns
)
#concatenation
X_test_df = pd.concat([X_test_scaled, X_test_ohe], axis=1)
X_test_df

c:\Users\Hp\anaconda3\envs\learn-env\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,amount_tsh,gps_height,longitude,latitude,region_code,district_code,population,construction_year,0,A/co Germany,...,dam,hand pump,improved spring,other,cattle trough,communal standpipe,dam,hand pump,improved spring,other
2980,0.000000,0.033174,0.793240,0.688925,0.163265,0.0625,0.0000,0.000000,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5246,0.000000,0.033174,0.814249,0.572276,0.132653,0.0750,0.0000,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
22659,0.000072,0.650571,0.880105,0.633074,0.204082,0.0125,0.0148,0.997516,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
39888,0.000000,0.033174,0.821888,0.216387,0.112245,0.0750,0.0000,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
13361,0.000362,0.441946,0.848579,0.616771,0.122449,0.0125,0.0235,0.999006,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17841,0.001449,0.687431,0.857796,0.153262,0.102041,0.0625,0.0530,0.994039,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
20222,0.003623,0.715076,0.908797,0.718680,0.010204,0.0250,0.0150,0.988574,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
40219,0.000000,0.033174,0.757487,0.784715,0.173469,0.3750,0.0000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19599,0.000000,0.269812,0.932341,0.475600,0.040816,0.0750,0.0001,0.995529,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [106]:
X_test_df.columns = X_test_df.columns.astype(str)

In [113]:
print(X_train_df.shape)  # Should be (n_samples_train, n_features)
print(X_test_df.shape)   # Should be (n_samples_test, n_features)


(47520, 3801)
(11880, 3801)


In [111]:
X_test_df = X_test_df.loc[:, ~X_test_df.columns.duplicated()]

In [110]:
# Check for duplicates in X_train_df
print(X_train_df.columns[X_train_df.columns.duplicated()])

# Check for duplicates in X_test_df
print(X_test_df.columns[X_test_df.columns.duplicated()])

Index([], dtype='object')
Index(['0', 'Adra', 'Africa Amini Alama', 'Amref', 'Anglican Church',
       'Ardhi Instute', 'Bahresa', 'Bao', 'Baric', 'Belgij',
       ...
       'spring', 'unknown', 'dam', 'other', 'cattle trough',
       'communal standpipe', 'dam', 'hand pump', 'improved spring', 'other'],
      dtype='object', length=262)


In [112]:
X_test_df = X_test_df.reindex(columns=X_train_df.columns, fill_value=0)

In [114]:
y_hat_test = logreg.predict(X_test_df)

test_residuals = np.abs(y_test - y_hat_test)
print(pd.Series(test_residuals, name="Residuals (counts)").value_counts())
print()
print(pd.Series(test_residuals, name="Residuals (proportions)").value_counts(normalize=True))

0    6366
1    3210
2    2304
Name: Residuals (counts), dtype: int64

0    0.535859
1    0.270202
2    0.193939
Name: Residuals (proportions), dtype: float64


### Evaluation Matrics

In [118]:
y_test.shape

(11880,)

In [119]:
y_hat_test.shape

(11880,)

In [120]:
# classification report 
from sklearn.metrics import classification_report

report = classification_report(y_true=y_test,y_pred=y_hat_test)

print(report)

              precision    recall  f1-score   support

           0       0.75      0.48      0.59      6457
           1       0.15      0.57      0.23       851
           2       0.63      0.61      0.62      4572

    accuracy                           0.54     11880
   macro avg       0.51      0.55      0.48     11880
weighted avg       0.66      0.54      0.57     11880



### SMOTE

In [125]:
y_train= pd.Series(y_train)

In [126]:
y_train.value_counts()

0    25802
2    18252
1     3466
dtype: int64

In [128]:
from imblearn.over_sampling import SMOTE

In [129]:
# Smote 
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_df,y_train)

y_train_smote.value_counts()

2    25802
1    25802
0    25802
dtype: int64

In [134]:
#model = LogisticRegression(max_iter=500)
#model_smote = model.fit(X_train_smote,y_train_smote)

c:\Users\Hp\anaconda3\envs\learn-env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


###  AUC

In [137]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

# Binarize the output (One-vs-Rest format)
classes = sorted(set(y_test))
y_test_bin = label_binarize(y_test, classes=classes)

# Get decision scores
y_score = model.decision_function(X_test_df)

# Calculate AUC for micro and macro averages
micro_auc = roc_auc_score(y_test_bin, y_score, average="micro")
macro_auc = roc_auc_score(y_test_bin, y_score, average="macro")

print(f"Micro-Averaged AUC: {micro_auc:.2f}")
print(f"Macro-Averaged AUC: {macro_auc:.2f}")


Micro-Averaged AUC: 0.77
Macro-Averaged AUC: 0.78


### Random Forest

In [139]:
from sklearn.ensemble import RandomForestClassifier
model_rfc = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model_rfc.fit(X_train_smote, y_train_smote)
predictions_rfc = model_rfc.predict(X_test_df)

In [140]:
report3 = classification_report(y_true=y_test,y_pred=predictions_rfc)
print(report3)

              precision    recall  f1-score   support

           0       0.74      0.59      0.66      6457
           1       0.17      0.51      0.26       851
           2       0.68      0.63      0.66      4572

    accuracy                           0.60     11880
   macro avg       0.53      0.58      0.52     11880
weighted avg       0.68      0.60      0.63     11880



In [142]:
# Get predicted probabilities for the test set
probabilities_rfc = model_rfc.predict_proba(X_test_df)

# Binarize the target variable for AUC calculation
classes = sorted(set(y_test))
y_test_rfc = label_binarize(y_test, classes=classes)

# Calculate micro- and macro-averaged AUC
micro_auc = roc_auc_score(y_test_rfc, probabilities_rfc, average="micro", multi_class="ovr")
macro_auc = roc_auc_score(y_test_rfc, probabilities_rfc, average="macro", multi_class="ovr")

print(f"Micro-Averaged AUC: {micro_auc:.2f}")
print(f"Macro-Averaged AUC: {macro_auc:.2f}")


Micro-Averaged AUC: 0.80
Macro-Averaged AUC: 0.77
